In [5]:
import abc

# 自定义的抽象基类要继承ABC
class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterable):
        # 从可迭代对象中添加元素
        pass
    @abc.abstractmethod
    def pick(self):
        # 随即删除元素并返回
        pass
    # 抽象类可以包含具体方法
    def loaded(self):
        # 如果至少有一个元素，返回True，否则返回False
        return bool(self.inspect())
    def inspect(self):
        # 返回一个有序元组，由当前元素构成
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [2]:
class Fake(Tombola):
    def pick(self):
        return 13
    
Fake

__main__.Fake

In [3]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

### 11.7.2 定义Tombola抽象基类的子类

In [6]:
import random

class BingoCage(Tombola):
    def __init__(self, items) -> None:
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    def __call__(self):
        self.pick()

In [7]:
class LotteryBlower(Tombola):
    def __init__(self, iterable) -> None:
        self._balls = list(iterable)
    def load(self, iterable):
        self._balls.extend(iterable)
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty lotteryBlower')
        return self._balls.pop(position)
    def loaded(self):
        return bool(self._balls)
    def inspect(self):
        return tuple(sorted(self._balls))

### 11.7.3 Tombola的虚拟子类

In [8]:
from random import randrange

@Tombola.register
class TomboList(list):
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
    load = list.extend
    def loaded(self):
        return bool(self)
    def inspect(self):
        return tuple(sorted(self))
# Tombola.register(TomboList)

In [9]:
issubclass(TomboList, Tombola)

True

In [10]:
t = TomboList(range(100))
isinstance(t, Tombola)

True